In [86]:
import socket
import random
import numpy as np
import matplotlib.pyplot as plt

In [68]:
def buildPacket(sync = 0, ack = 0, fin = 0, seqNum = 0, ackNum = 0, payload = 0, payloadSize = 0, divisor = "000000000"):
    flags = f"000{ack}00{sync}{fin}"
    stringPacket = flags + f"{seqNum:032b}" + f"{ackNum:032b}" + f"{payloadSize:016b}" + f"{payload:01024b}"
    CRC = ""
    if int(divisor) > 0:
        CRC = getCRC(stringPacket, divisor)

    else:
        CRC = "00000000"
    stringPacket += CRC
    
    listPacket = np.array([val for val in stringPacket]).reshape(int(np.ceil(len(stringPacket)/8)),8)
    intPacket = [int("".join(listByte), base = 2) for listByte in listPacket]

    return intPacket

In [69]:
def getCRC(binCode, divisor):
    payload = list(binCode) + ([0]*(len(divisor)-1))
    payload = [int(n) for n in payload]
    divisor = [int(n) for n in list(divisor)]
    remainder = []
    pointer = 0

    while pointer <= len(payload) - len(divisor):
        #Extract slice to subtract
        val = payload[pointer: pointer + len(divisor)]
        
        #Subtract paired values
        operations = zip(val, divisor)
        remainder = [abs(op[0] - op[1]) for op in operations]
        
        #Modify payload values, set pointer to index
        for i in range(len(divisor)):
            payload[pointer + i] = remainder[i]
        
        #Move pointer to next non-zero value
        if 1 not in payload:
            return "0"*(len(divisor)-1)
        pointer = payload.index(1)

    return "".join([str(val) for val in payload[-(len(divisor) - 1):]])

In [70]:
def reconstructHeader(packet, low, high):
    headerVal = int("".join([f"{packByte:08b}" for packByte in packet[low:high]]), base = 2)
    return headerVal

In [71]:
def separateData(packet):
    stringPacket = [f"{packByte:08b}" for packByte in packet]
    flags = stringPacket[0]
    seqNum = int("".join(stringPacket[1:5]), base = 2)
    ackNum = int("".join(stringPacket[5:9]), base = 2)
    payloadSize = int("".join(stringPacket[9:11]), base = 2)
    payload = "".join(stringPacket[11:-1])[-payloadSize:]
    return (flags, seqNum, ackNum, payload)


In [72]:
def decipherMessage(message):
    msgBytes = np.array([char for char in message]).reshape(int(len(message)/8), 8)
    letters = [chr(int("".join(msgByte), base = 2)) for msgByte in msgBytes]
    return "".join(letters)

In [82]:
def decipherImage(message):
    msgBytes = [str(int(msg)) for msg in message]
    msgBytes = "".join(msgBytes)
    msgNum = int(msgBytes, base = 2)
    hexNum = hex(msgNum)[2:]
    if len(hexNum)%2 != 0:
        hexNum += "0"
    data = bytes.fromhex(hexNum)

    return data

In [74]:
def checkCRC(binCode, divisor):
    stringPacket = [f"{packByte:08b}" for packByte in binCode]
    payload = []
    for byteSection in stringPacket:
        payload += byteSection
    payload = [int(n) for n in payload]
    divisor = [int(n) for n in list(divisor)]
    remainder = []
    pointer = 0

    while pointer <= len(payload) - len(divisor):
        #Extract slice to subtract
        val = payload[pointer: pointer + len(divisor)]
        
        #Subtract paired values
        operations = zip(val, divisor)
        remainder = [abs(op[0] - op[1]) for op in operations]
        
        #Modify payload values, set pointer to index
        for i in range(len(divisor)):
            payload[pointer + i] = remainder[i]
        
        #Move pointer to next non-zero value
        if 1 not in payload:
            return 0
        pointer = payload.index(1)

    return "".join([str(val) for val in payload[-(len(divisor) - 1):]])

In [90]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(("localhost", 8000))
    s.listen()
    while True:
        cnt = 0
        conn, adr = s.accept()
        with conn:
            print(f"Connected to {adr}")

            seqNum = 0
            ackNum = 1
            otherDivisor = "100000111"
            totalPayload = ""

            # Receive Sync Request
            syncReq = conn.recv(2048)
            sync = int(f"{syncReq[0]:08b}"[6])

            if sync:
                print("Client wants to synchronize")
            else:
                conn.close()
                break

            # Accept Sync Request
            ack = buildPacket(sync=1, ack=1, ackNum=ackNum, seqNum=seqNum)
            seqNum += 1
            conn.send(bytes(ack))

            # Receive Sync Ack
            syncAck = int(f"{conn.recv(2048)[0]:08b}"[3])
            if (syncAck):
                print("Ready to receive data")

            else:
                conn.close()
                break            



            while True:
                failRate = random.random()
                
                # Receive Data
                print("waiting for data")
                packet = conn.recv(2048)
                print("processing data")
                clFlags, clSeqNum, clAckNum, payloadPiece = separateData(packet)

                if failRate < 0.2:
                    print("Random Packet Error")
                    stringPacket = [f"{packByte:08b}" for packByte in packet]
                    bit1 = int(stringPacket[-2])
                    bit2 = int(stringPacket[-1])
                    stringPacket[-1] = stringPacket[-1][:-2] + str(bit1^1) + str(bit2^1)
                    packet = [int("".join(listByte), base = 2) for listByte in stringPacket]




                fin = int(clFlags[-1])
                crcCheck = checkCRC(packet, otherDivisor)

                print(crcCheck)
                
                if fin:
                    print("Client is finished sending data")
                    conn.close()
                    break
                
                if crcCheck == 0:
                    ackNum += len(packet)
                    totalPayload += payloadPiece

                else:
                    print(crcCheck,len(payloadPiece),len(otherDivisor))
                    print("".join([f"{packByte:08b}" for packByte in packet]))
                    print("Erroneous data, try to request packet")
                
                
                # Send Acknowledgement
                print("ack", ackNum)
                ack = buildPacket(ack=1, ackNum=ackNum, seqNum=seqNum)
                conn.send(bytes(ack))

                print(f"cnt: {cnt} >>> {int(payloadPiece)}")
                cnt += 1



        print("Displaying data")
        imgPayload = decipherImage(totalPayload)
        savedImage = plt.imread("recImage.jpg")
        plt.imshow(savedImage)

        with open('recImage.jpg', 'wb') as jpg:
            jpg.write(imgPayload)
        break


Connected to ('127.0.0.1', 8088)
Client wants to synchronize
Ready to receive data
waiting for data
processing data
0
ack 141
cnt: 0 >>> 11111111110110001111111111100000000000000001000001001010010001100100100101000110000000000000000100000001000000100000000000111011000000000011101100000000000000001111111111011011000000000100001100000000000001010000001100000100000001000000010000000011000001010000010000000100000001000000010100000101000001010000011000000111000011000000100000000111000001110000011100000111000011110000101100001011000010010000110000010001000011110001001000010010000100010000111100010001000100010001001100010110000111000001011100010011000101000001101000010101000100010001000100011000001000010001100000011010000111010001110100011111000111110001111100010011000101110010001000100100001000100001111000100100000111000001111000011111000111101111111111011011000000000100001100000001000001010000010100000101000001110000011000000111000011100000100000001000000011100001111000010100000100010001010

IndexError: list index out of range